In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDGN
import tensorflow.keras as ks
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from PIL import Image, ImageDraw
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)
    # for filename in filenames:
    #     print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions


/kaggle/input
/kaggle/input/lacuna-malaria-detection-dataset
/kaggle/input/lacuna-malaria-detection-dataset/images


In [2]:
# /kaggle/input/lacuna-malaria-detection-dataset/Train.csv
# /kaggle/input/lacuna-malaria-detection-dataset/Test.csv
#ImageDraw

# Train Data

In [3]:
df = pd.read_csv('/kaggle/input/lacuna-malaria-detection-dataset/Train.csv')

In [4]:
df

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,id_u3q6jdck4j.jpg,Trophozoite,1.0,712,1241,737,1270
1,id_a6cl90trri.jpg,Trophozoite,1.0,558,1566,600,1604
2,id_qvc2le9sm8.jpg,Trophozoite,1.0,1317,2788,1448,2914
3,id_w8xnbd5rvm.jpg,Trophozoite,1.0,925,1744,1041,1823
4,id_6dop09rk02.jpg,NEG,1.0,0,0,0,0
...,...,...,...,...,...,...,...
23525,id_vxfv34g53d.jpg,WBC,1.0,2336,2441,2537,2647
23526,id_61axoyt1e6.jpg,WBC,1.0,486,713,543,770
23527,id_ipmba1040m.jpg,Trophozoite,1.0,456,497,486,535
23528,id_r2x9t2l5cd.jpg,Trophozoite,1.0,2974,1039,3094,1165


In [5]:
df['class'].unique()

array(['Trophozoite', 'NEG', 'WBC'], dtype=object)

In [6]:
df['bin_classes'] = df['class'].apply(lambda x : '0' if x == 'NEG' else '1' )

In [7]:
df['bin_classes'].unique()

array(['1', '0'], dtype=object)

Extracting image paths using Image Ids

In [8]:
df['img_path']=  df['Image_ID'].apply(lambda x : os.path.join('/kaggle/input/lacuna-malaria-detection-dataset/images/',x))

Checking if the image exist

In [9]:
df['Exists']=  df['img_path'].apply(lambda x :True if os.path.exists(x) else False)

In [10]:
df['Exists'].unique()

array([ True])

In [11]:
# test_Df['bin_classes'] = test_Df['class'].apply(lambda x : 0 if x == 'NEG' else 1 )

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23530 entries, 0 to 23529
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Image_ID     23530 non-null  object 
 1   class        23530 non-null  object 
 2   confidence   23530 non-null  float64
 3   ymin         23530 non-null  int64  
 4   xmin         23530 non-null  int64  
 5   ymax         23530 non-null  int64  
 6   xmax         23530 non-null  int64  
 7   bin_classes  23530 non-null  object 
 8   img_path     23530 non-null  object 
 9   Exists       23530 non-null  bool   
dtypes: bool(1), float64(1), int64(4), object(4)
memory usage: 1.6+ MB


In [13]:
df.isnull().sum()

Image_ID       0
class          0
confidence     0
ymin           0
xmin           0
ymax           0
xmax           0
bin_classes    0
img_path       0
Exists         0
dtype: int64

In [14]:
df.head(10)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,bin_classes,img_path,Exists
0,id_u3q6jdck4j.jpg,Trophozoite,1.0,712,1241,737,1270,1,/kaggle/input/lacuna-malaria-detection-dataset...,True
1,id_a6cl90trri.jpg,Trophozoite,1.0,558,1566,600,1604,1,/kaggle/input/lacuna-malaria-detection-dataset...,True
2,id_qvc2le9sm8.jpg,Trophozoite,1.0,1317,2788,1448,2914,1,/kaggle/input/lacuna-malaria-detection-dataset...,True
3,id_w8xnbd5rvm.jpg,Trophozoite,1.0,925,1744,1041,1823,1,/kaggle/input/lacuna-malaria-detection-dataset...,True
4,id_6dop09rk02.jpg,NEG,1.0,0,0,0,0,0,/kaggle/input/lacuna-malaria-detection-dataset...,True
5,id_i7r9hkhxvx.jpg,WBC,1.0,846,906,903,963,1,/kaggle/input/lacuna-malaria-detection-dataset...,True
6,id_fzogq9kijh.jpg,WBC,1.0,641,280,711,373,1,/kaggle/input/lacuna-malaria-detection-dataset...,True
7,id_i8qcpi8ey9.jpg,Trophozoite,1.0,501,1141,532,1170,1,/kaggle/input/lacuna-malaria-detection-dataset...,True
8,id_y0me081u0v.jpg,NEG,1.0,0,0,0,0,0,/kaggle/input/lacuna-malaria-detection-dataset...,True
9,id_2phbgf1x5m.jpg,Trophozoite,1.0,687,1169,725,1205,1,/kaggle/input/lacuna-malaria-detection-dataset...,True


# Test DB

In [16]:
test_Df = pd.read_csv('/kaggle/input/lacuna-malaria-detection-dataset/Test.csv')

In [21]:
test_Df

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,bin_classes,img_path,Exists
0,id_5n9ov0rr22.jpg,NaN,NaN,NaN,NaN,NaN,NaN,1,/kaggle/input/lacuna-malaria-detection-dataset...,True
1,id_k6s11k3klo.jpg,NaN,NaN,NaN,NaN,NaN,NaN,1,/kaggle/input/lacuna-malaria-detection-dataset...,True
2,id_eftvcf9djb.jpg,NaN,NaN,NaN,NaN,NaN,NaN,1,/kaggle/input/lacuna-malaria-detection-dataset...,True
3,id_kc5tmoj1xa.jpg,NaN,NaN,NaN,NaN,NaN,NaN,1,/kaggle/input/lacuna-malaria-detection-dataset...,True
4,id_tirid4qt68.jpg,NaN,NaN,NaN,NaN,NaN,NaN,1,/kaggle/input/lacuna-malaria-detection-dataset...,True
...,...,...,...,...,...,...,...,...,...,...
1173,id_btrtdkgk4r.jpg,NaN,NaN,NaN,NaN,NaN,NaN,1,/kaggle/input/lacuna-malaria-detection-dataset...,True
1174,id_straufuobm.jpg,NaN,NaN,NaN,NaN,NaN,NaN,1,/kaggle/input/lacuna-malaria-detection-dataset...,True
1175,id_nnurq35wvp.jpg,NaN,NaN,NaN,NaN,NaN,NaN,1,/kaggle/input/lacuna-malaria-detection-dataset...,True
1176,id_hdqd25rput.jpg,NaN,NaN,NaN,NaN,NaN,NaN,1,/kaggle/input/lacuna-malaria-detection-dataset...,True


In [22]:
train_df, val_df = train_test_split(df ,test_size =0.2, stratify=df['bin_classes'])

In [23]:
data_gen = IDGN( preprocessing_function= preprocess_input, rescale  = 1./255,validation_split=0.2)
# test_datagen = IDGN(zoom_range = 0.2, shear_range = 0.2, horizontal_flip = True,preprocessing_function= preprocess_input)
test_datagen = IDGN(zoom_range = 0.2, shear_range = 0.2, horizontal_flip = True,preprocessing_function= preprocess_input)

In [50]:
train_data  = data_gen.flow_from_dataframe(dataframe = df ,
                                           x_col = 'img_path',
                                           y_col = 'bin_classes',
                                           target_size=(224, 224),
                                           color_mode = 'rgb' ,
                                           class_mode  = 'binary',
                                           dtype = 'float32',
                                           subset = 'training',
                                           batch_size  = 32, shuffle =True)

Found 18824 validated image filenames belonging to 2 classes.


In [51]:
val_data = data_gen.flow_from_dataframe(dataframe = val_df , 
                                             directory = '/kaggle/input/lacuna-malaria-detection-dataset/images',
                                             x_col = 'img_path',
                                             y_col = 'bin_classes',
                                             color_mode = 'rgb' ,
                                        subset = 'validation',
                                        target_size=(224, 224),
                                             class_mode  = 'binary',
                                        dtype = 'float32',
                                             batch_size  = 32,
                                             shuffle =True)

Found 941 validated image filenames belonging to 2 classes.


In [29]:
for i in range(5):  # Check first few batches
    batch_x, batch_y = next(train_data)
    print(f"Batch {i+1}: x_dtype={batch_x.dtype}, y_dtype={batch_y.dtype}")

Batch 1: x_dtype=float32, y_dtype=float32
Batch 2: x_dtype=float32, y_dtype=float32
Batch 3: x_dtype=float32, y_dtype=float32
Batch 4: x_dtype=float32, y_dtype=float32
Batch 5: x_dtype=float32, y_dtype=float32


In [27]:
for i in range(5):  # Check first few batches
    batch_x, batch_y = next(val_data)
    print(f"Batch {i+1}: x_dtype={batch_x.dtype}, y_dtype={batch_y.dtype}")

Batch 1: x_dtype=float32, y_dtype=object
Batch 2: x_dtype=float32, y_dtype=object
Batch 3: x_dtype=float32, y_dtype=object
Batch 4: x_dtype=float32, y_dtype=object
Batch 5: x_dtype=float32, y_dtype=object


In [41]:
from  keras.layers import Dense, Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19
import keras

In [52]:
base_model = VGG19(input_shape= (224, 224,3), include_top=False)

In [53]:
for layer in base_model.layers:
  layer.trainable = False

In [54]:
base_model.summary()

Model: "vgg19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv4 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv4 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv4 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,024,384 (76.39 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 20,024,384 (76.39 MB)

In [55]:
from keras.layers import  Dropout, Conv2D,BatchNormalization

In [56]:
from keras.regularizers import l2

In [57]:
X =Conv2D(32, (3, 3), activation='relu')(base_model.output)
X = BatchNormalization()(X)
X = Dropout(0.5)(X)
x = Conv2D(64, (3, 3), activation='relu')(X)
X= Flatten()(X)
X = Dense(units  = 64,kernel_regularizer=l2(0.001), activation =  'relu')(X)
X = Dense(units  = 32,kernel_regularizer=l2(0.001), activation =  'relu')(X)
X = Dense(units  = 1,kernel_regularizer=l2(0.01), activation =  'sigmoid')(X)
model  =  Model(inputs = [base_model.input], outputs = [X])

In [58]:
model.compile(optimizer  = 'adam', loss =  keras.losses.binary_crossentropy, metrics = ['accuracy'])

In [59]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

ErStop = EarlyStopping(monitor = 'val_accuracy', min_delta = 0.1, patience  = 3 , verbose = 1)
ModCheck = ModelCheckpoint(filepath = '/kaggle/best_model.keras',
                           monitor = 'accuracy',
                           verbose = 1,
                           save_best_only = True)

In [60]:
cb = [ErStop,ModCheck]

In [61]:
history  =  model.fit(train_data,
                               steps_per_epoch = 32,
                               epochs = 50,
                               verbose = 1,
                               callbacks = cb,
                               validation_data = val_data,
                               validation_steps = 32
                     )

Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8154 - loss: 0.6531
Epoch 1: accuracy improved from -inf to 0.93555, saving model to /kaggle/best_model.keras


/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


32/32 ━━━━━━━━━━━━━━━━━━━━ 104s 3s/step - accuracy: 0.8191 - loss: 0.6437 - val_accuracy: 0.9957 - val_loss: 0.2670
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9919 - loss: 0.1665
Epoch 2: accuracy improved from 0.93555 to 0.99512, saving model to /kaggle/best_model.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accuracy: 0.9920 - loss: 0.1663 - val_accuracy: 0.9989 - val_loss: 0.1839
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 1.0000 - loss: 0.1401
Epoch 3: accuracy improved from 0.99512 to 1.00000, saving model to /kaggle/best_model.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 69s 2s/step - accuracy: 1.0000 - loss: 0.1400 - val_accuracy: 1.0000 - val_loss: 0.1406
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 911ms/step - accuracy: 1.0000 - loss: 0.1301
Epoch 4: accuracy did not improve from 1.00000
32/32 ━━━━━━━━━━━━━━━━━━━━ 67s 2s/step - accuracy: 1.0000 - loss: 0.1301 - val_accuracy: 1.0000 - val_loss: 0.1262
Epoch 4: early stopping
